# Local Spatial Functions

Local spatial functions compute a variety of spatial information on a local, per tile basis, meaning that every tile is evaluated independently. `withCenter`, `withCenterLatLng`, `withBounds`, and `withSpatialIndex` all create a new column in the rasterframe titled `center`, `center`, `bounds`, and `spatial_index` respectively containing the results of the query.

In [ ]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import geotrellis.spark.SpatialKey._
import geotrellis.raster._
import geotrellis.raster.mapalgebra.local._


implicit val spark = SparkSession.builder().
  master("local").appName("RasterFrames").
  config("spark.ui.enabled", "false").
  getOrCreate().
  withRasterFrames

val sc = spark.sparkContext

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")
val bandNumbers = 1 to 4
val filenamePattern = "L8-B%d-Elkton-VA.tiff"

In [ ]:
val joinedRF = bandNumbers.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ (b, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"band_$b") }.
  reduce(_ spatialJoin _)

`withCenter` creates a column containing the center of the tiles in the native CRS.

In [8]:
joinedRF.withCenter().select("center").show(false)

+---------------------------------------+
|center                                 |
+---------------------------------------+
|POINT (706767.7980160001 4252076.74344)|
+---------------------------------------+



`withCenterLatLng` performs a similar function, but displays the centers in longitude, latitude form (EPSG:4326).

In [11]:
joinedRF.withCenterLatLng().select("center").show(false)

+--------------------------------------+
|center                                |
+--------------------------------------+
|[-78.63240544923785,38.39301867116852]|
+--------------------------------------+



`withBounds` creates a column containing the polygonal bounds of the tile in the native CRS.

In [13]:
joinedRF.withBounds().select("bounds").show(false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|bounds                                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|POLYGON ((703986.502389 4249551.61978, 703986.502389 4254601.8671, 709549.093643 4254601.8671, 709549.093643 4249551.61978, 703986.502389 4249551.61978))|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+



`withSpatialIndex` creates a spatial index column
# What does this mean

In [15]:
joinedRF.withSpatialIndex().select("spatial_index").show(false)

+-------------------+
|spatial_index      |
+-------------------+
|2781571893406213788|
+-------------------+

